# Partial Differential Equations
Introductory approach on analytical solutions, numerical approximations, and error analysis of PDEs.

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import numpy as np

## 1d Heat Equation

The heat equation in one dimension is
$$\dfrac{\partial u}{\partial t} = \dfrac{\partial^2u}{\partial x^2},$$
where $u(t, x)$ is a function of time $t$ and space $x$, and has boundary conditions $u\vert_{x = 0} = u\vert_{x = L} = 0$, and initial condition $u(x, t = 0) = \phi(x)$. The general solution to this pde is
$$u = \sum_{n = 1}^\infty a_ne^{-\lambda_n^2t}\sin(\lambda_nx),$$
where $\lambda_n = n\pi/L$, and
$$a_n = \dfrac{2}{L}\int_0^L\phi(x)\sin(\lambda_nx)dx.$$
Set $L = 1$ and $\phi(x) = 1$. Then $\lambda_n = n\pi$
$$a_n = 2\int_0^1\sin(n\pi x)dx = \dfrac{1 - \cos(n\pi)}{n\pi} = \begin{cases}
\dfrac{2}{n\pi}&n\textrm{ odd},\\
0 &\textrm{else}.
\end{cases}$$
Let $n = 2k-1$, then the specific solution is
$$u = \sum_{k = 1}^\infty\dfrac{2}{(2k-1)\pi}e^{-(2k-1)^2\pi^2t}\sin[(2k-1)\pi x].$$

In [ ]:
pi = np.pi
exp = np.exp
sin = np.sin

def heat_equation_terms(t, x, k):
    # returns the kth term in the series solution at time t and space x.
    return 2/((2*k-1)*pi)*exp(-(2*k-1)**2*pi**2*t)*sin((2*k-1)*pi*x)

def heat_equation_solve(t, x, N):
    # returns value of u at t, x with N terms in series.
    if isinstance(x, np.ndarray):
        return np.array([heat_equation_solve(t, _x, N) for _x in x])
    if isinstance(t, np.ndarray):
        return np.array([heat_equation_solve(_t, x, N) for _t in t])
    
    return sum(heat_equation_terms(t, x, k) for k in range(1, N+1))

In [ ]:
n, m = 100, 100

tt = np.linspace(0, 1, n)
xx = np.linspace(0, 1, m)
T, X = np.meshgrid(tt, xx)
U = heat_equation_solve(tt, xx, 100)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.contourf(T, X, U, cmap = 'magma')
ax.set_xlabel('t')
ax.set_ylabel('x')
ax.set_title('Heat Equation in 1D')
plt.show()

### Explicit Scheme
The algorithm for the explicit scheme is
$$u_j^{i+1} = \lambda u_{j+1}^i + (1 - 2\lambda)u_j^i + \lambda u_{j-1}^i,$$
where $\lambda = \Delta t/\Delta x^2$.

In [ ]:
n, m = 145, 10

tt = np.linspace(0, 1, n)
xx = np.linspace(0, 1, m)
T, X = np.meshgrid(tt, xx)

t_min, t_max = tt[0], tt[n-1]
dt = (t_max - t_min) / (n - 1)

x_min, x_max = xx[0], xx[m-1]
dx = (x_max - x_min) / (m - 1)

# compute lambda.
lamb = dt/np.power(dx, 2)
print(lamb)

# empty solution array.
U = np.array([[.0 for _ in range(m)] for _ in range(n)])

for i in range(n):
    if i == 0:
        U[0] = np.array([1. for _ in range(m)])
        pass
    else:
        for j in range(m):
            if j == 0 or j == m - 1:
                U[i][j] = 0
                pass
            else:
                U[i][j] = lamb*U[i-1][j-1] + (1 - 2*lamb)*U[i-1][j] + lamb*U[i-1][j+1]
                pass
            pass
        pass
    pass

U = np.transpose(U)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.contourf(T, X, U, cmap = 'magma')
ax.set_title('Heat Equation (Explicit Scheme)')
plt.show()

*Note*: $\lambda$ must be sufficiently small (i.e. $\lambda\ll 1$ or $\Delta t\ll\Delta x$), otherwise the solution will diverge. (Explain why having large timesteps $\Delta t$ would be problematic?)

In [ ]:
def heat_equation_explicit_scheme(tt, xx):
    n, m = len(tt), len(xx)
    t_min, t_max = tt[0], tt[n-1]
    x_min, x_max = xx[0], xx[m-1]
    dt, dx = (t_max - t_min)/(n-1), (x_max - x_min)/(m-1)
    
    lamb = dt/np.power(dx, 2)
    
    U = np.array([[0. for _ in range(m)] for _ in range(n)])
    for i in range(n):
        if i == 0:
            U[0] = np.array([1. for _ in range(m)])
            pass
        else:
            for j in range(m):
                if j == 0 or j == m-1:
                    U[i][j] = 0.
                    pass
                else:
                    U[i][j] = lamb*U[i-1][j-1] + (1 - 2*lamb)*U[i-1][j] + lamb*U[i-1][j+1]
                    pass
                pass
            pass
        pass
    return np.transpose(U), lamb

### Stability

We find a reasonably accurate solution when $\lambda<.54$. In fact, the explicit scheme is stable if and only if $\lambda\leq 1/2$, termed *conditionally stable*. ([Davis](http://wwwf.imperial.ac.uk/~mdavis/FDM11/LECTURE_SLIDES2.PDF))

In [ ]:
nn = [10, 140, 145, 150, 160, 170]

m = 10
xx = np.linspace(0, 1, m)

plt.subplots(len(nn), 1, figsize = (6.4, 4.8*len(nn)))

for i, n in enumerate(nn):
    tt = np.linspace(0, 1, n)
    solution, lamb = heat_equation_explicit_scheme(tt, xx)
    T, X = np.meshgrid(tt, xx)
    
    plt.subplot(len(nn), 1, i+1)
    plt.contourf(T, X, solution, cmap = 'magma')
    plt.title('n = {}, lambda = {}'.format(n, lamb))
    pass
plt.show()